# Connect to the OCHIN DB through R

This notebook will walk you through how to connect to the OCHIN DB using R. 
Before you begin, make sure that you have access to the data and check to make sure the `db-credentials.txt` file is located in your home directory. 

## Install ODBC drivers as necessary

In [3]:
system("sudo su -c 'curl https://packages.microsoft.com/config/rhel/7/prod.repo > /etc/yum.repos.d/mssql-release.repo && exit'")
system("sudo yum remove unixODBC-utf16 unixODBC-utf16-devel")
system("sudo ACCEPT_EULA=Y yum install -y msodbcsql17")
system("sudo ACCEPT_EULA=Y yum install -y mssql-tools")
system("echo 'export PATH='$PATH:/opt/mssql-tools/bin'' >> ~/.bashrc")
system("source ~/.bashrc")
system("sudo yum install -y unixODBC-devel")

## Install and load R packages

In [4]:
install.packages("odbc")
library(odbc)
install.packages("DBI")
install.packages("rstudioapi")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



## Read and parse your DB credentials

In [5]:
file_path <- '/home/ec2-user/SageMaker/db-credentials.txt'

db_creds_df <- read.table(file_path)
db_creds <- db_creds_df$V3
names(db_creds) <- db_creds_df$V1

db_creds <- trimws(db_creds)
#db_creds

## Connect to the database
Remember that you may have access to multiple databases. 


### Connecting to the "raw" data
The raw database contains the patient data. You have read-only access to this database.

- For AIM AHEAD Year 1 programs, the Raw database is named FellowsSample.
- For AIM AHEAD Year 2 programs, the Raw database is named AAOCHIN2023.
- For AIM AHEAD Year 3 programs, the Raw database is named AAOCHIN2024.

Within the Raw database, you must open the **View** specific to your project. Your OCHIN Project ID is provided to you when you receive confirmation of your OCHIN DB Login Activation. 

Once you have opened your View, you will see the Tables which contain the subset of data specific to your project. 

- Tables containing patient-level data specific to your project will be prefixed with your OCHIN Project ID.
- Tables that contain metadata shared amongst all projects are prefixed with "common".

In [7]:
### This example shows how to view tables in the AAOCHIN2024 database. 
raw_db <- 'AAOCHIN2023' 
raw_connection_string = paste0("DRIVER={ODBC Driver 17 for SQL Server};",
                          "SERVER=", db_creds['host'], ',', db_creds['port'], ';',
                          "DATABASE=", raw_db, ';',
                          "UID=", db_creds['username'], ';',
                          "PWD={", db_creds['password'], "};")
raw_db_conn <- DBI::dbConnect(odbc::odbc(), .connection_string = raw_connection_string)

In [8]:
### We can print all tables available in the database views we have access to like so: 
df1 <- dbGetQuery(raw_db_conn, "SELECT name AS TABLE_NAME FROM sys.views;")
df1

TABLE_NAME
<chr>
OBSERVATION_FACT
PATIENT_DIMENSION
PROVIDER_DIMENSION
VISIT_DIMENSION
ZCTA_CVS
SIDECAR_FACILITIES
CONCEPT_DIMENSION
I2B2


In [9]:
### We can view the top 10 entries in the CONCEPT_DIMENSION table like so:
### Note that the CONCEPT_DIMENSION table contains metadata shared amongst all projects, 
### and therefore must be prefixed with "common"
df2 <- dbGetQuery(raw_db_conn, "SELECT TOP 10 * FROM common.CONCEPT_DIMENSION;")
df2

CONCEPT_PATH,CONCEPT_CD,NAME_CHAR
<chr>,<chr>,<chr>
\i2b2\Demographics\Age\0-9 years old\0 years old\01 months\,DEM|AGE:0.1,01 months
\i2b2\Demographics\Age\0-9 years old\0 years old\02 months\,DEM|AGE:0.2,02 months
\i2b2\Demographics\Age\0-9 years old\0 years old\03 months\,DEM|AGE:0.3,03 months
\i2b2\Demographics\Age\0-9 years old\0 years old\04 months\,DEM|AGE:0.4,04 months
\i2b2\Demographics\Age\0-9 years old\0 years old\05 months\,DEM|AGE:0.5,05 months
\i2b2\Demographics\Age\0-9 years old\0 years old\06 months\,DEM|AGE:0.6,06 months
\i2b2\Demographics\Age\0-9 years old\0 years old\07 months\,DEM|AGE:0.7,07 months
\i2b2\Demographics\Age\0-9 years old\0 years old\08 months\,DEM|AGE:0.8,08 months
\i2b2\Demographics\Age\0-9 years old\0 years old\09 months\,DEM|AGE:0.9,09 months


We can then save this data to our SageMaker workspace for use later
Data saved as CSV files in this way can be retrieved at a later date, so you would not have to re-write SQL code

In [11]:
# Creates a folder named "data_files" that you can use to organize your files
#os.mkdir("data_files")

# Save the results from our SQL query to the data_files folder
write.csv(df2,"data_files/results.csv")


In order to retrieve previous data, we can read in the CSV file we saved to use as a dataframe for further analysis

In [12]:
previous_results = read.csv("data_files/results.csv")
previous_results

X,CONCEPT_PATH,CONCEPT_CD,NAME_CHAR
<int>,<chr>,<chr>,<chr>
1,\i2b2\Demographics\Age\0-9 years old\0 years old\01 months\,DEM|AGE:0.1,01 months
2,\i2b2\Demographics\Age\0-9 years old\0 years old\02 months\,DEM|AGE:0.2,02 months
3,\i2b2\Demographics\Age\0-9 years old\0 years old\03 months\,DEM|AGE:0.3,03 months
4,\i2b2\Demographics\Age\0-9 years old\0 years old\04 months\,DEM|AGE:0.4,04 months
5,\i2b2\Demographics\Age\0-9 years old\0 years old\05 months\,DEM|AGE:0.5,05 months
6,\i2b2\Demographics\Age\0-9 years old\0 years old\06 months\,DEM|AGE:0.6,06 months
7,\i2b2\Demographics\Age\0-9 years old\0 years old\07 months\,DEM|AGE:0.7,07 months
8,\i2b2\Demographics\Age\0-9 years old\0 years old\08 months\,DEM|AGE:0.8,08 months
9,\i2b2\Demographics\Age\0-9 years old\0 years old\09 months\,DEM|AGE:0.9,09 months


### Connecting to your project database
Your project database is unique to your project and named after your OCHIN Project ID. You have read-write access to this database.

Your project database is initially empty. All temporary tables or aggregate results for your project should be saved here. You can store up to 50GB in your project database. This is not a lot! The full Raw database has over 1TB of data. If you copy data directly from the Raw database to your project database, you will quickly run out of space.



In [ ]:
### This example shows how to view tables in your project specific database. 
proj_db <- 'S000' # Change this to your project ID
proj_connection_string = paste0("DRIVER={ODBC Driver 17 for SQL Server};",
                          "SERVER=", db_creds['host'], ',', db_creds['port'], ';',
                          "DATABASE=", proj_db, ';',
                          "UID=", db_creds['username'], ';',
                          "PWD={", db_creds['password'], "};")
proj_db_conn <- DBI::dbConnect(odbc::odbc(), .connection_string = proj_connection_string)

In [ ]:
### We can print all tables in the project database like so:
### Note that this database will be empty unless you have saved tables here. 
df3 <- dbGetQuery(proj_db_conn, "SELECT name AS TABLE_NAME FROM sys.tables;")
df3